# Дипломный проект Некрасов А.В.

Импортируем необходимые билиотеки

In [66]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
#from mpl_toolkits.basemap import Basemap
#from sklearn import metrics, linear_model
#from sklearn.preprocessing import PolynomialFeatures, StandardScaler
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
#from scipy.optimize import curve_fit
#plt.rcParams["patch.force_edgecolor"] = True
#plt.style.use('fivethirtyeight')
#mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
#%matplotlib inline
warnings.filterwarnings("ignore")

1 Найти аэропорт с минимальной задержкой вылета 

2 Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

3 Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

4 Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета  

In [67]:
df = pd.read_csv('flights.csv', low_memory=False)

In [68]:
tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100)
                         .T.rename(index={0:'null values (%)'}))
tab_info

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
column type,int64,int64,int64,int64,object,int64,object,object,object,int64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,object,float64,float64,float64,float64,float64
null values (nb),0,0,0,0,0,0,14721,0,0,0,86153,86153,89047,89047,6,105071,105071,0,92513,92513,0,92513,105071,0,0,5729195,4755640,4755640,4755640,4755640,4755640
null values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.252978,0.0,0.0,0.0,1.480526,1.480526,1.530259,1.530259,0.000103,1.805629,1.805629,0.0,1.589822,1.589822,0.0,1.589822,1.805629,0.0,0.0,98.455357,81.72496,81.72496,81.72496,81.72496,81.72496


In [69]:
df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,2355.0,-4.0,22.0,17.0,320.0,298.0,272.0,2611,749.0,4.0,819,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,2355.0,-4.0,17.0,12.0,227.0,215.0,195.0,1617,427.0,3.0,446,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,2350.0,-9.0,17.0,7.0,221.0,222.0,197.0,1598,424.0,8.0,440,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,2353.0,-6.0,10.0,3.0,161.0,157.0,144.0,1189,327.0,3.0,340,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
airports = pd.read_csv('airports.csv')
airports

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [163]:
l_airports_df = pd.read_csv('L_AIRPORT.csv')
l_airports_id_df = pd.read_csv('L_AIRPORT_ID.csv')

In [187]:
l_airports_df

,Code,Description
0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,04A,"Lik, AK: Lik Mining Camp"
3,05A,"Little Squaw, AK: Little Squaw Airport"
4,06A,"Kizhuyak, AK: Kizhuyak Bay"
...,...,...
6424,ZXZ,"Waterville, WA: Waterville Airport"
6425,ZYL,"Sylhet, Bangladesh: Savannakhet"
6426,ZZU,"Mzuzu, Malawi: Mzuzu Airport"
6427,ZZV,"Zanesville, OH: Zanesville Municipal"


In [167]:
airports_code_df = pd.concat([l_airports_df, l_airports_id_df], axis=0)

In [168]:
airports_code_df

,Code,Description
0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,04A,"Lik, AK: Lik Mining Camp"
3,05A,"Little Squaw, AK: Little Squaw Airport"
4,06A,"Kizhuyak, AK: Kizhuyak Bay"
...,...,...
6409,16748,"Doylestown, PA: Doylestown Airport"
6410,16749,"China Spring, TX: Wildcat Canyon"
6411,16750,"Sanford, NC: Raleigh Exec Jetport at Sanford L..."
6412,16751,"Placida, FL: Coral Creek"


In [158]:
airports_code_df

,Code_x,Description,Code_y
0,01A,"Afognak Lake, AK: Afognak Lake Airport",10001
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip",10003
2,04A,"Lik, AK: Lik Mining Camp",10004
3,05A,"Little Squaw, AK: Little Squaw Airport",10005
4,06A,"Kizhuyak, AK: Kizhuyak Bay",10006
...,...,...,...
6438,ZXZ,"Waterville, WA: Waterville Airport",16358
6439,ZYL,"Sylhet, Bangladesh: Savannakhet",16359
6440,ZZU,"Mzuzu, Malawi: Mzuzu Airport",16360
6441,ZZV,"Zanesville, OH: Zanesville Municipal",16361


In [161]:
airports_full_inf_df = airports.merge(airports_code_df, left_on='IATA_CODE', right_on='Code_x', how='left')

In [162]:
airports_full_inf_df

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,Code_x,Description,Code_y
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",10135
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190,ABI,"Abilene, TX: Abilene Regional",10136
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,ABQ,"Albuquerque, NM: Albuquerque International Sun...",10140
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183,ABR,"Aberdeen, SD: Aberdeen Regional",10141
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447,ABY,"Albany, GA: Southwest Georgia Regional",10146
...,...,...,...,...,...,...,...,...,...,...
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764,WYS,"West Yellowstone, MT: Yellowstone",15897
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681,XNA,"Fayetteville, AR: Northwest Arkansas Regional",15919
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023,YAK,"Yakutat, AK: Yakutat Airport",15991
321,YUM,Yuma International Airport,Yuma,AZ,USA,32.65658,-114.60597,YUM,"Yuma, AZ: Yuma MCAS/Yuma International",13785


In [71]:
airlines = pd.read_csv('airlines.csv')
airlines

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


,Code_x,Description,Code_y
0,01A,"Afognak Lake, AK: Afognak Lake Airport",10001
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip",10003
2,04A,"Lik, AK: Lik Mining Camp",10004
3,05A,"Little Squaw, AK: Little Squaw Airport",10005
4,06A,"Kizhuyak, AK: Kizhuyak Bay",10006
...,...,...,...
6438,ZXZ,"Waterville, WA: Waterville Airport",16358
6439,ZYL,"Sylhet, Bangladesh: Savannakhet",16359
6440,ZZU,"Mzuzu, Malawi: Mzuzu Airport",16360
6441,ZZV,"Zanesville, OH: Zanesville Municipal",16361


In [72]:
df['DATE'] = pd.to_datetime(df[['YEAR','MONTH', 'DAY']])

In [73]:
#_________________________________________________________
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure
#_____________________________________________________________________
# Function that combines a date and time to produce a datetime.datetime
def combine_date_heure(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
#_______________________________________________________________________________
# Function that combine two columns of the dataframe to create a datetime format
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_heure(cols))
        else:
            cols[1] = format_heure(cols[1])
            liste.append(combine_date_heure(cols))
    return pd.Series(liste)

In [75]:
df['SCHEDULED_DEPARTURE'] = create_flight_time(df, 'SCHEDULED_DEPARTURE')
df['DEPARTURE_TIME'] = df['DEPARTURE_TIME'].apply(format_heure)
df['SCHEDULED_ARRIVAL'] = df['SCHEDULED_ARRIVAL'].apply(format_heure)
df['ARRIVAL_TIME'] = df['ARRIVAL_TIME'].apply(format_heure)
df['WHEELS_OFF'] = df['WHEELS_OFF'].apply(format_heure)
df['WHEELS_ON'] = df['WHEELS_ON'].apply(format_heure)

In [77]:
df.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME',
             'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'TAXI_OUT','TAXI_IN', 'WHEELS_OFF', 'WHEELS_ON', 'ELAPSED_TIME']]

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY,TAXI_OUT,TAXI_IN,WHEELS_OFF,WHEELS_ON,ELAPSED_TIME
0,2015-01-01 00:05:00,04:30:00,23:54:00,04:08:00,-11.0,-22.0,21.0,4.0,00:15:00,04:04:00,194.0
1,2015-01-01 00:10:00,07:50:00,00:02:00,07:41:00,-8.0,-9.0,12.0,4.0,00:14:00,07:37:00,279.0
2,2015-01-01 00:20:00,08:06:00,00:18:00,08:11:00,-2.0,5.0,16.0,11.0,00:34:00,08:00:00,293.0
3,2015-01-01 00:20:00,08:05:00,00:15:00,07:56:00,-5.0,-9.0,15.0,8.0,00:30:00,07:48:00,281.0
4,2015-01-01 00:25:00,03:20:00,00:24:00,02:59:00,-1.0,-21.0,11.0,5.0,00:35:00,02:54:00,215.0
5,2015-01-01 00:25:00,06:02:00,00:20:00,06:10:00,-5.0,8.0,18.0,6.0,00:38:00,06:04:00,230.0


In [78]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,CANCELLATION_REASON,5729195,1.544643
1,LATE_AIRCRAFT_DELAY,4755640,18.275040
2,AIRLINE_DELAY,4755640,18.275040
3,SECURITY_DELAY,4755640,18.275040
4,AIR_SYSTEM_DELAY,4755640,18.275040
5,WEATHER_DELAY,4755640,18.275040
6,ELAPSED_TIME,105071,98.194371
7,AIR_TIME,105071,98.194371
8,ARRIVAL_DELAY,105071,98.194371
9,WHEELS_ON,92513,98.410178


variables_to_remove = ['YEAR', 
                       'MONTH','DAY_OF_WEEK', 'AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIR_TIME']
df.drop(variables_to_remove, axis = 1, inplace = True)
df = df[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
        'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
        'SCHEDULED_TIME', 'ELAPSED_TIME']]

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 32 columns):
 #   Column               Dtype         
---  ------               -----         
 0   YEAR                 int64         
 1   MONTH                int64         
 2   DAY                  int64         
 3   DAY_OF_WEEK          int64         
 4   AIRLINE              object        
 5   FLIGHT_NUMBER        int64         
 6   TAIL_NUMBER          object        
 7   ORIGIN_AIRPORT       object        
 8   DESTINATION_AIRPORT  object        
 9   SCHEDULED_DEPARTURE  datetime64[ns]
 10  DEPARTURE_TIME       object        
 11  DEPARTURE_DELAY      float64       
 12  TAXI_OUT             float64       
 13  WHEELS_OFF           object        
 14  SCHEDULED_TIME       float64       
 15  ELAPSED_TIME         float64       
 16  AIR_TIME             float64       
 17  DISTANCE             int64         
 18  WHEELS_ON            object        
 19  TAXI_IN              

# Найти аэропорт с минимальной задержкой вылета

In [140]:
delay_airport_df = df[['ORIGIN_AIRPORT', 'DEPARTURE_DELAY']]

In [179]:
delay_airport_df

,ORIGIN_AIRPORT,DEPARTURE_DELAY
0,ANC,-11.0
1,LAX,-8.0
2,SFO,-2.0
3,LAX,-5.0
4,SEA,-1.0
...,...,...
5819074,LAX,-4.0
5819075,JFK,-4.0
5819076,JFK,-9.0
5819077,MCO,-6.0


In [178]:
airports_code_df = pd.concat([l_airports_df, l_airports_id_df], axis=0)

In [180]:
airports_code_df

,Code,Description
0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,04A,"Lik, AK: Lik Mining Camp"
3,05A,"Little Squaw, AK: Little Squaw Airport"
4,06A,"Kizhuyak, AK: Kizhuyak Bay"
...,...,...
6409,16748,"Doylestown, PA: Doylestown Airport"
6410,16749,"China Spring, TX: Wildcat Canyon"
6411,16750,"Sanford, NC: Raleigh Exec Jetport at Sanford L..."
6412,16751,"Placida, FL: Coral Creek"


In [188]:
delay_airport_code_df = delay_airport_df.merge(airports_code_df, left_on= 'ORIGIN_AIRPORT', right_on='Code', how='inner')

In [189]:
delay_airport_code_df

,ORIGIN_AIRPORT,DEPARTURE_DELAY,Code,Description
0,ANC,-11.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
1,ANC,-4.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
2,ANC,-14.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
3,ANC,-4.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
4,ANC,NaN,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
...,...,...,...,...
5332909,HYA,16.0,HYA,"Hyannis, MA: Barnstable Municipal-Boardman/Pol..."
5332910,HYA,-11.0,HYA,"Hyannis, MA: Barnstable Municipal-Boardman/Pol..."
5332911,HYA,0.0,HYA,"Hyannis, MA: Barnstable Municipal-Boardman/Pol..."
5332912,HYA,-13.0,HYA,"Hyannis, MA: Barnstable Municipal-Boardman/Pol..."


In [218]:
positive_delay_airport_code_df = delay_airport_code_df.loc[delay_airport_code_df['DEPARTURE_DELAY']>0]

positive_delay_airport_code_df

,ORIGIN_AIRPORT,DEPARTURE_DELAY,Code,Description
19,ANC,4.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
28,ANC,40.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
32,ANC,39.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
38,ANC,15.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
42,ANC,8.0,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati..."
...,...,...,...,...
5332893,HYA,13.0,HYA,"Hyannis, MA: Barnstable Municipal-Boardman/Pol..."
5332894,HYA,6.0,HYA,"Hyannis, MA: Barnstable Municipal-Boardman/Pol..."
5332896,HYA,21.0,HYA,"Hyannis, MA: Barnstable Municipal-Boardman/Pol..."
5332904,HYA,8.0,HYA,"Hyannis, MA: Barnstable Municipal-Boardman/Pol..."


In [221]:
mean_positive_delay_airport_code_df = positive_delay_airport_code_df.groupby('Code').mean()

In [222]:
mean_positive_delay_airport_code_df

,DEPARTURE_DELAY
Code,
ABE,44.519672
ABI,44.254937
ABQ,32.059219
ABR,63.171429
ABY,51.926471
...,...
WRG,36.067010
WYS,24.243902
XNA,51.078333


In [223]:
median_positive_delay_airport_code_df = positive_delay_airport_code_df.groupby('Code').median()

In [224]:
median_positive_delay_airport_code_df

,DEPARTURE_DELAY
Code,
ABE,16.0
ABI,19.0
ABQ,15.0
ABR,18.0
ABY,28.5
...,...
WRG,17.0
WYS,8.0
XNA,23.0


In [194]:
mean_delay_airport_code_df = mean_delay_airport_code_df.merge(l_airports_df, left_on= 'Description', right_on='Description', how='inner')

In [199]:
mean_delay_airport_code_df.sort_values('DEPARTURE_DELAY')

,Description,DEPARTURE_DELAY,Code
306,"Vernal, UT: Vernal Regional",-11.933333,VEL
321,"Yakutat, AK: Yakutat Airport",-6.138686,YAK
202,"Moab, UT: Canyonlands Field",-5.250000,CNY
47,"Butte, MT: Bert Mooney",-4.704698,BTM
175,"Lewiston, ID: Lewiston Nez Perce County",-4.130435,LWS
...,...,...,...
124,"Gustavus, AK: Gustavus Airport",20.052632,GST
74,"Crescent City, CA: Jack McNamara Field",20.675325,CEC
67,"Columbia, MO: Columbia Regional",22.107692,COU
190,"Martha's Vineyard, MA: Martha's Vineyard Airport",26.726316,MVY
